# Module - Benchmarking
Ontime provides a `Benchmark` class that can be used to run a number of prediction models on a number of datasets.

In [1]:
from ontime.module.benchmarking import BenchmarkMode, BenchmarkDataset, BenchmarkMetric, BenchmarkModelConfig, Benchmark
import ontime as on

## Initialization
A `Benchmark` instance can be initialized with a list of datasets, models and metrics to run through. When invoking `run()`, it will train (if needed) and test every dataset on every model, and compute every metric on the predicted data.


### Preparing datasets

Datasets submitted to a `Benchmark` must be of type `TimeSeries`, wrapped into `BenchmarkDataset`. `BenchmarkDataset` allows to give datasets a name, give training and test splits, and define how data will be split to perform a rolling evaluation.

In [2]:
from ontime.module.datasets.dataset import Dataset
from darts.utils.missing_values import fill_missing_values # for filling missing values in the time series (for models that don't handle missing values)

datasets = [
    BenchmarkDataset(on.TimeSeries.from_darts(fill_missing_values(Dataset.TemperatureDataset.load())), input_length=96, target_length=24, gap=0, stride=96, name="Daily temperature"),
]

### Preparing models  

Benchmark models must be given in a `BenchmarkModelConfig` class object, and their class must implement ontime `AbstractModel` interface. The `BenchmarkModelConfig` class contains attribute that allows to later instantiate the model with the desired configuration.
It requires the following parameters :
- `model_name` : the name of the model, for result logging purpose,
- `model_class` : the class of the model that implements `AbstractModel` interface,
- `benchmark_mode` : the mode in which the model must be evaluted:
    - if `ZERO_SHOT`, the model is not trained, and the evaluation is done on the test set. It is used for models that already has trained weights, available through checkpoints, or for some models from darts, where predictions are directly made using the fitted data as input (such as ARIMA),
    - if `FULL_SHOT`, the model is trained on the entire given training set. Once trained, the model is evaluated using the learnt weights.
- `static_model_params` : the static parameters to give to the model class for instantiating it. This parameters can be defined when instatiating the `BenchmarkModelConfig` object.
- `dynamic_model_params` : the dynamic parameters to give to the model class for instantiating it. This parameters can only be known when the dataset on which the model is trained is known. Therefore, a callable object that take a `BenchmarkDataset` must be given.

In [3]:
from ontime.core.time_series.time_series import TimeSeries
from ontime import Model
from darts.models import ExponentialSmoothing, TCNModel

# torch related parameters
pl_trainer_kwargs = {
    "accelerator": "cpu",
    "enable_progress_bar": False
    }

# dynamic parameters callback
input_length_param = lambda ds: ds.input_length
target_length_param = lambda ds: ds.target_length


model_configs = [
    BenchmarkModelConfig("ExponentialSmoothing", model_class=Model, benchmark_mode=BenchmarkMode.ZERO_SHOT, static_model_params={"model" : ExponentialSmoothing()}),
    BenchmarkModelConfig("Temporal Convolutional Network", model_class=Model, benchmark_mode=BenchmarkMode.FULL_SHOT, 
                         static_model_params={"model":TCNModel, "n_epochs":2, "pl_trainer_kwargs":pl_trainer_kwargs}, 
                         dynamic_model_params={"input_chunk_length":input_length_param, "output_chunk_length":target_length_param})
]

### Preparing metrics
Metrics must be given to the `BenchmarkMetric` constructor. If the function can't be invoked as is in `BenchmarkMetric`'s implementation, a child class can be written and submitted.

In [4]:
import darts.metrics

metrics = [
   BenchmarkMetric(name="MAE", metric_function=darts.metrics.metrics.mae),
   BenchmarkMetric(name="sMAPE", metric_function=darts.metrics.metrics.smape),
   BenchmarkMetric(name="MASE", metric_function=darts.metrics.metrics.mase)
]

## Creating and running a Benchmark

In [5]:
benchmark = Benchmark(datasets=datasets,
                      model_configs=model_configs, 
                      metrics=metrics)

Datasets, models and metrics can also be added after instanciation. This allows to name datasets.

In [6]:
benchmark.add_dataset(BenchmarkDataset(Dataset.ETTh1Dataset.load()[:5000], input_length=336, gap=0, stride=72, target_length=72, name = "ETTh1"))

Once the models and datasets have been added, the `run()` method will train instances of all the models on all the datasets individually and compute metrics. The logging level can be chosen to show less or more information about the benchmark execution in the console.

In [7]:
benchmark.run(logging_level="debug")

on 0: 08:34:31 - [INFO] - BenchmarkLogger - On Daily temperature dataset...


on 1: 08:34:31 - [INFO] - BenchmarkLogger - ExponentialSmoothing model...


on 1: 08:34:31 - [INFO] - BenchmarkLogger - Evaluating...


on 1: 08:34:31 - [INFO] - BenchmarkLogger - Evaluation done, took 0.530205249786377


on 1: 08:34:31 - [INFO] - BenchmarkLogger - getting predictions...


...

on 4: lightning.pytorch.utilities.rank_zero INFO  TPU available: False, using: 0 TPU cores


on 4: INFO: HPU available: False, using: 0 HPUs


on 4: lightning.pytorch.utilities.rank_zero INFO  HPU available: False, using: 0 HPUs


on 4: 08:34:59 - [INFO] - BenchmarkLogger - Computed metrics: 
       {'MAE': 2.529447460260427, 'sMAPE': 99.82058944577555, 'MASE': 4.438016277794369}


Benchmarking |████████████████████| 4/4 [100%] in 28.9s (0.14/s)                                                                                                                                        


## Visualizing results

The benchmark automatically stores measures and metrics computed during the run, available through class attributes.

### Measures and metrics
To view the results, you can call `get_report()` and print the returned value

In [8]:
print(benchmark.get_report())

Daily temperature dataset:
nb features: 1
target column: ['Daily minimum temperatures']
training set size: 2335
validation set size: 585
test set size: 732

ExponentialSmoothing model:
suceeded: ✓
training time: 0
evaluation time: 0.530205249786377
inference time: 0.06768155097961426
metrics: {'MAE': 2.400040758215776, 'sMAPE': 24.148347520589876, 'MASE': 1.259049717016714}

Temporal Convolutional Network model:
suceeded: ✓
training time: 4.2949488162994385
evaluation time: 0.5235123634338379
inference time: 0.3829374313354492
metrics: {'MAE': 2.441540289454, 'sMAPE': 24.977305752818385, 'MASE': 1.2361577158238866}


ETTh1 dataset:
nb features: 7
target column: ['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']
training set size: 3198
validation set size: 801
test set size: 1001

ExponentialSmoothing model:
suceeded: ✓
training time: 0
evaluation time: 11.291918516159058
inference time: 1.0528616905212402
metrics: {'MAE': 1.6361909709419278, 'sMAPE': 54.35492758310376, 'MASE': 2.40

You can also get results as dataframes by calling `get_report_df()`. The results are then returned as a dataframe with model names as columns, dataset names as main rows, and measure as sub rows.

In [9]:
df_1, df_2 = benchmark.get_report_dfs()
df_1

,Daily temperature,ETTh1
Characteristic,,
nb features,1,7
target column,[Daily minimum temperatures],"[HUFL, HULL, MUFL, MULL, LUFL, LULL, OT]"
training set size,2335,3198
validation set size,585,801
test set size,732,1001


In [10]:
df_2

ExponentialSmoothing  \
Dataset           Metric                                  
Daily temperature training time                0.000000   
                  evaluation time              0.530205   
                  inference time               0.067682   
                  MAE                          2.400041   
                  sMAPE                       24.148348   
                  MASE                         1.259050   
ETTh1             training time                0.000000   
                  evaluation time             11.291919   
                  inference time               1.052862   
                  MAE                          1.636191   
                  sMAPE                       54.354928   
                  MASE                         2.405468   

                                   Temporal Convolutional Network  
Dataset           Metric                                           
Daily temperature training time                          4.294949  
                  evaluation time                        0.523512  
                  inference time                         0.382937  
                  MAE                                    2.441540  
                  sMAPE                                 24.977306  
                  MASE                                   1.236158  
ETTh1             training time                          9.150265  
                  evaluation time                        0.692194  
                  inference time                         0.342503  
                  MAE                                    2.529447  
                  sMAPE                                 99.820589  
                  MASE                                   4.438016

### Plotting

By default (argument `nb_predictions` of `benchmark.run()` method), the benchmark will generate a prediction for one random input sample of each dataset with each model. The predictions, along input and target series, are stored in a dictionnary and can be retrieved by calling `benchmark.get_predictions()`. The predictions can be plotted using the Ontime plotting module.

In [11]:
predictions = benchmark.get_predictions()

In [12]:
# currently, Ontime plotting module needs the time index to be named 'time'
def rename_index(ts, name='time'):
    df = ts.pd_dataframe()
    df.rename_axis(name, inplace=True)
    return TimeSeries.from_dataframe(df)

In [13]:
input = rename_index(predictions['inputs']['Daily temperature'][0]).rename({'Daily minimum temperatures': 'input'})
target = rename_index(predictions['targets']['Daily temperature'][0]).rename({'Daily minimum temperatures': 'target'})
prediction = rename_index(predictions['predictions']['Daily temperature']['Temporal Convolutional Network'][0]).rename({'Daily minimum temperatures': 'prediction'})

In [14]:
(on.Plot()
    .add(on.marks.line, input)
    .add(on.marks.line, target)
    .add(on.marks.line, prediction, type='dashed')
    .properties(width=600, height=200)
    .show()
)

alt.LayerChart(...)

In [15]:
prediction = rename_index(predictions['predictions']['Daily temperature']['ExponentialSmoothing'][0]).rename({'Daily minimum temperatures': 'prediction'})

In [16]:
(on.Plot()
    .add(on.marks.line, input)
    .add(on.marks.line, target)
    .add(on.marks.line, prediction, type='dashed')
    .properties(width=600, height=200)
    .show()
)

alt.LayerChart(...)

In [17]:
input = rename_index(predictions['inputs']['ETTh1'][0][500:].univariate_component(0)).rename({'HUFL': 'input'})
target = rename_index(predictions['targets']['ETTh1'][0].univariate_component(0)).rename({'HUFL': 'target'})
prediction = rename_index(predictions['predictions']['ETTh1']['ExponentialSmoothing'][0].univariate_component(0)).rename({'HUFL': 'prediction'})

In [18]:
(on.Plot()
    .add(on.marks.line, input)
    .add(on.marks.line, target)
    .add(on.marks.line, prediction, type='dashed')
    .properties(width=600, height=200)
    .show()
)

alt.LayerChart(...)